In [ ]:
# ============================================================
# CONFIGURACIÓN E IMPORTS
# ============================================================

import pandas as pd
import numpy as np
import re
import os
from datetime import datetime
from pathlib import Path
import sys

# Agregar la raíz del proyecto al path
ROOT = Path().resolve().parent
sys.path.insert(0, str(ROOT))

# Importar configuración centralizada
from config import PATHS, CONFIG

print("✓ Configuración cargada correctamente")

In [ ]:
# Usar rutas centralizadas
CARPETA_LIMPIA = PATHS.PROCESSED
CARPETA_FEATURES = PATHS.FEATURES
ARCHIVO_ENTRADA = PATHS.PUBLICACIONES_TEXTO

# Usar configuración centralizada
USAR_PYSENTIMIENTO = CONFIG.USAR_PYSENTIMIENTO
USAR_EMOJIS = CONFIG.USAR_EMOJIS
MOSTRAR_PROGRESO = CONFIG.MOSTRAR_PROGRESO
PROGRESO_CADA = CONFIG.PROGRESO_CADA

print("="*60)
print("CONFIGURACIÓN ACTIVA:")
print("="*60)
print(f"Archivo entrada: {ARCHIVO_ENTRADA}")
print(f"Carpeta salida: {CARPETA_FEATURES}")
print(f"Pysentimiento: {'Activo' if USAR_PYSENTIMIENTO else 'Inactivo'}")
print(f"Análisis de emojis: {'Activo' if USAR_EMOJIS else 'Inactivo'}")
print(f"Mostrar progreso cada {PROGRESO_CADA} registros")
print("="*60)

In [7]:
def extraer_caracteristicas_linguisticas(texto):
    """Extrae características lingüísticas básicas del texto"""
    
    palabras = texto.split()
    num_palabras = len(palabras)
    num_caracteres = len(texto)
    
    # Longitud promedio de palabras
    longitud_palabra_prom = np.mean([len(p) for p in palabras]) if num_palabras > 0 else 0
    
    # Contar signos de puntuación
    num_signos_exclamacion = texto.count('!')
    num_signos_pregunta = texto.count('?')
    num_puntos = texto.count('.')
    num_comas = texto.count(',')
    
    # Palabras únicas (riqueza léxica)
    palabras_unicas = len(set(palabras)) if num_palabras > 0 else 0
    riqueza_lexica = palabras_unicas / num_palabras if num_palabras > 0 else 0
    
    return {
        'num_palabras': num_palabras,
        'num_caracteres': num_caracteres,
        'longitud_palabra_prom': round(longitud_palabra_prom, 2),
        'num_signos_exclamacion': num_signos_exclamacion,
        'num_signos_pregunta': num_signos_pregunta,
        'num_puntos': num_puntos,
        'num_comas': num_comas,
        'palabras_unicas': palabras_unicas,
        'riqueza_lexica': round(riqueza_lexica, 3)
    }


def extraer_pronombres(texto):
    """Extrae métricas de uso de pronombres"""
    
    texto_lower = ' ' + texto.lower() + ' '
    num_palabras = len(texto.split())
    
    # Pronombres de primera persona singular
    pronombres_1era_sing = ['yo', 'me', 'mi', 'mí', 'conmigo']
    count_1era_sing = sum(texto_lower.count(f' {p} ') for p in pronombres_1era_sing)
    
    # Pronombres de primera persona plural
    pronombres_1era_plur = ['nosotros', 'nosotras', 'nos', 'nuestro', 'nuestra']
    count_1era_plur = sum(texto_lower.count(f' {p} ') for p in pronombres_1era_plur)
    
    # Pronombres de segunda persona
    pronombres_2da = ['tú', 'tu', 'te', 'ti', 'contigo', 'usted', 'ustedes']
    count_2da = sum(texto_lower.count(f' {p} ') for p in pronombres_2da)
    
    # Pronombres de tercera persona
    pronombres_3era = ['él', 'ella', 'ellos', 'ellas', 'le', 'les', 'lo', 'la', 'los', 'las']
    count_3era = sum(texto_lower.count(f' {p} ') for p in pronombres_3era)
    
    return {
        'num_pronombres_1era_sing': count_1era_sing,
        'num_pronombres_1era_plur': count_1era_plur,
        'num_pronombres_2da': count_2da,
        'num_pronombres_3era': count_3era,
        'pct_pronombres_1era_sing': round((count_1era_sing / max(num_palabras, 1)) * 100, 2),
        'pct_pronombres_1era_plur': round((count_1era_plur / max(num_palabras, 1)) * 100, 2),
        'pct_pronombres_2da': round((count_2da / max(num_palabras, 1)) * 100, 2),
        'pct_pronombres_3era': round((count_3era / max(num_palabras, 1)) * 100, 2)
    }


def extraer_palabras_clave(texto):
    """Extrae métricas de palabras clave indicadoras"""
    
    texto_lower = texto.lower()
    num_palabras = len(texto.split())
    
    # Palabras absolutistas
    absolutistas = ['siempre', 'nunca', 'todo', 'nada', 'todos', 'nadie', 'ninguno', 'jamás']
    count_absolutistas = sum(texto_lower.count(p) for p in absolutistas)
    
    # Palabras negativas
    negativas = ['no', 'ni', 'sin', 'nunca', 'nada', 'nadie']
    count_negativas = sum(texto_lower.count(f' {p} ') for p in negativas)
    
    # Palabras de causalidad
    causales = ['porque', 'causa', 'razón', 'debido', 'por eso', 'por lo tanto']
    count_causales = sum(texto_lower.count(p) for p in causales)
    
    # Palabras tentativas/incertidumbre
    tentativas = ['quizás', 'tal vez', 'posiblemente', 'probablemente', 'quizá', 'puede']
    count_tentativas = sum(texto_lower.count(p) for p in tentativas)
    
    return {
        'num_absolutistas': count_absolutistas,
        'num_negativas': count_negativas,
        'num_causales': count_causales,
        'num_tentativas': count_tentativas,
        'pct_absolutistas': round((count_absolutistas / max(num_palabras, 1)) * 100, 2),
        'pct_negativas': round((count_negativas / max(num_palabras, 1)) * 100, 2),
        'pct_causales': round((count_causales / max(num_palabras, 1)) * 100, 2),
        'pct_tentativas': round((count_tentativas / max(num_palabras, 1)) * 100, 2)
    }


def extraer_emojis(texto_original):
    """Extrae métricas de emojis (requiere librería emoji)"""
    
    if not USAR_EMOJIS:
        return {
            'num_emojis': 0,
            'emojis_por_palabra': 0
        }
    
    try:
        import emoji
        num_emojis = emoji.emoji_count(texto_original)
        num_palabras = len(texto_original.split())
        
        return {
            'num_emojis': num_emojis,
            'emojis_por_palabra': round(num_emojis / max(num_palabras, 1), 3)
        }
    except ImportError:
        print("Librería 'emoji' no instalada. Instalar con: pip install emoji")
        return {
            'num_emojis': 0,
            'emojis_por_palabra': 0
        }


def extraer_sentimiento_emocion(texto):
    """Extrae análisis de sentimiento y emoción usando pysentimiento"""
    
    if not USAR_PYSENTIMIENTO:
        return {
            'sentimiento': None,
            'score_pos': 0,
            'score_neg': 0,
            'score_neu': 0,
            'emocion': None,
            'score_alegria': 0,
            'score_tristeza': 0,
            'score_enojo': 0,
            'score_miedo': 0,
            'score_sorpresa': 0,
            'score_disgusto': 0
        }
    
    try:
        from pysentimiento import create_analyzer
        
        # Crear analizadores (se cachean automáticamente)
        if not hasattr(extraer_sentimiento_emocion, 'analyzer_sent'):
            extraer_sentimiento_emocion.analyzer_sent = create_analyzer(task="sentiment", lang="es")
            extraer_sentimiento_emocion.analyzer_emo = create_analyzer(task="emotion", lang="es")
        
        # Análisis de sentimiento
        sent = extraer_sentimiento_emocion.analyzer_sent.predict(texto)
        
        # Análisis de emoción
        emo = extraer_sentimiento_emocion.analyzer_emo.predict(texto)
        
        return {
            'sentimiento': sent.output,
            'score_pos': round(sent.probas.get('POS', 0), 4),
            'score_neg': round(sent.probas.get('NEG', 0), 4),
            'score_neu': round(sent.probas.get('NEU', 0), 4),
            'emocion': emo.output,
            'score_alegria': round(emo.probas.get('joy', 0), 4),
            'score_tristeza': round(emo.probas.get('sadness', 0), 4),
            'score_enojo': round(emo.probas.get('anger', 0), 4),
            'score_miedo': round(emo.probas.get('fear', 0), 4),
            'score_sorpresa': round(emo.probas.get('surprise', 0), 4),
            'score_disgusto': round(emo.probas.get('disgust', 0), 4)
        }
    except ImportError:
        print("Librería 'pysentimiento' no instalada. Instalar con: pip install pysentimiento")
        return {
            'sentimiento': None,
            'score_pos': 0,
            'score_neg': 0,
            'score_neu': 0,
            'emocion': None,
            'score_alegria': 0,
            'score_tristeza': 0,
            'score_enojo': 0,
            'score_miedo': 0,
            'score_sorpresa': 0,
            'score_disgusto': 0
        }


def extraer_todas_caracteristicas(df):
    """
    Extrae TODAS las características de cada publicación
    
    Args:
        df: DataFrame con columnas 'texto_limpio' y 'texto_publicacion'
    
    Returns:
        DataFrame con todas las características extraídas
    """
    
    print(f"\n{'='*80}")
    print("EXTRACCIÓN DE CARACTERÍSTICAS")
    print(f"{'='*80}")
    print(f"Total de registros a procesar: {len(df)}")
    
    if USAR_PYSENTIMIENTO:
        print("✓ Análisis de sentimiento y emoción: ACTIVADO")
    else:
        print("Análisis de sentimiento y emoción: DESACTIVADO")
    
    if USAR_EMOJIS:
        print("✓ Análisis de emojis: ACTIVADO")
    else:
        print("Análisis de emojis: DESACTIVADO")
    
    print(f"\nProcesando...")
    
    resultados = []
    
    for idx, row in df.iterrows():
        texto_limpio = row.get('texto_limpio', '')
        texto_original = row.get('texto_publicacion', texto_limpio)
        
        # Extraer todas las características
        caracteristicas = {
            'id_participante': row.get('id_participante', ''),
            'id_publicacion': row.get('id_publicacion', ''),
            'fecha_publicacion': row.get('fecha_publicacion', '')
        }
        
        # 1. Características lingüísticas
        caracteristicas.update(extraer_caracteristicas_linguisticas(texto_limpio))
        
        # 2. Pronombres
        caracteristicas.update(extraer_pronombres(texto_limpio))
        
        # 3. Palabras clave
        caracteristicas.update(extraer_palabras_clave(texto_limpio))
        
        # 4. Emojis
        caracteristicas.update(extraer_emojis(texto_original))
        
        # 5. Sentimiento y emoción
        caracteristicas.update(extraer_sentimiento_emocion(texto_limpio))
        
        resultados.append(caracteristicas)
        
        # Mostrar progreso
        if MOSTRAR_PROGRESO and (idx + 1) % PROGRESO_CADA == 0:
            porcentaje = ((idx + 1) / len(df)) * 100
            print(f"  Procesadas {idx + 1}/{len(df)} publicaciones ({porcentaje:.1f}%)")
    
    print(f"\n✓ Todas las publicaciones procesadas")
    
    return pd.DataFrame(resultados)


def analizar_publicaciones():
    """Proceso completo de extracción de características"""
    
    print("="*80)
    print("ANÁLISIS DE CARACTERÍSTICAS DE PUBLICACIONES")
    print("="*80)
    
    # Crear carpeta de features si no existe
    if not os.path.exists(CARPETA_FEATURES):
        os.makedirs(CARPETA_FEATURES)
        print(f"✓ Carpeta '{CARPETA_FEATURES}/' creada")
    
    # Verificar archivo de entrada
    ruta_entrada = os.path.join(CARPETA_LIMPIA, ARCHIVO_ENTRADA)
    
    if not os.path.exists(ruta_entrada):
        print(f"\n Error: No se encuentra el archivo '{ruta_entrada}'")
        print(f"   Asegúrate de ejecutar primero el preprocesamiento")
        return None
    
    # Leer datos
    print(f"\nLeyendo archivo: {ruta_entrada}")
    try:
        df = pd.read_csv(ruta_entrada, encoding='utf-8-sig')
        print(f"✓ {len(df)} registros cargados")
    except Exception as e:
        print(f" Error leyendo archivo: {e}")
        return None
    
    # Verificar columnas necesarias
    columnas_necesarias = ['texto_limpio']
    if not all(col in df.columns for col in columnas_necesarias):
        print(f" Error: Falta la columna 'texto_limpio'")
        return None
    
    # Extraer características
    df_caracteristicas = extraer_todas_caracteristicas(df)
    
    # Guardar resultados
    nombre_salida = f"caracteristicas_completas.csv"
    ruta_salida = os.path.join(CARPETA_FEATURES, nombre_salida)
    df_caracteristicas.to_csv(ruta_salida, index=False, encoding='utf-8-sig')
    
    print(f"\n{'='*80}")
    print("✓ ANÁLISIS COMPLETADO")
    print(f"{'='*80}")
    print(f"Total de características extraídas: {len(df_caracteristicas.columns)} columnas")
    print(f"Archivo guardado: {ruta_salida}")
    
    # Mostrar estadísticas
    print(f"\n{'='*80}")
    print("ESTADÍSTICAS DE CARACTERÍSTICAS")
    print(f"{'='*80}")
    
    if 'sentimiento' in df_caracteristicas.columns and df_caracteristicas['sentimiento'].notna().any():
        print("\nDistribución de sentimientos:")
        print(df_caracteristicas['sentimiento'].value_counts())
        
        print("\nDistribución de emociones:")
        print(df_caracteristicas['emocion'].value_counts())
    
    print("\nPromedio de características lingüísticas:")
    cols_numericas = ['num_palabras', 'num_caracteres', 'longitud_palabra_prom', 
                      'riqueza_lexica', 'pct_pronombres_1era_sing']
    for col in cols_numericas:
        if col in df_caracteristicas.columns:
            print(f"  {col}: {df_caracteristicas[col].mean():.2f}")
    
    return df_caracteristicas


In [ ]:
if __name__ == "__main__":
    df_resultado = analizar_publicaciones()